# L4: Process Inputs: Chain of Thought Reasoning

## Configuration
#### Chargement de la clé OpenAI et des librairies Python

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)


In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

## Chain-of-Thought Prompting

In [3]:
delimiter = "####"
system_message = f"""
Suivez ces étapes pour répondre aux demandes des clients.
La demande du client sera délimitée par quatre hashtags, c'est-à-dire {delimiter}.

Étape 1:{delimiter} Décidez d'abord si l'utilisateur
pose une question sur un produit spécifique ou sur des produits.
La catégorie de produits ne compte pas.

Étape 2:{delimiter} Si l'utilisateur pose des questions sur des 
produits spécifiques, identifiez si les produits figurent dans 
la liste suivante. Tous les produits disponibles : 
1. Produit : TechPro Ultrabook
   Catégorie : Computers and Laptops
   Marque : TechPro
   Numéro du modèle : TP-UB100
   Garantie : 1 year
   Note : 4,5
   Caractéristiques : écran de 13"3, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description : Un ultrabook élégant et léger pour une utilisation quotidienne.
   Prix : 799,99€

2. Produit : BlueWave Gaming Laptop
   Catégorie : Computers and Laptops
   Marque : BlueWave
   Numéro du modèle : BW-GL200
   Garantie : 2 years
   Notes : 4,7
   Caractéristiques : écran de 15"6, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description : Un ordinateur portable de jeu haute performance pour une expérience immersive.
   Prix : 1199,99€

3. Produit : PowerLite Convertible
   Catégorie : Computers and Laptops
   Marque : PowerLite
   Numéro du modèle : PL-CV300
   Garantie : 1 year
   Note : 4,3
   Caractéristiques : écran de 14", 8GB RAM, 256GB SSD, 360-degree hinge
   Description : Un ordinateur portable convertible polyvalent avec un écran tactile réactif.
   Prix : 699,99€

4. Produit : TechPro Desktop
   Catégorie : Computers and Laptops
   Marque : TechPro
   Numéro du modèle : TP-DT500
   Garantie : 1 year
   Note : 4,4
   Caractéristiques : Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: Un ordinateur de bureau puissant pour le travail et les loisirs.
   Prix : 999,99€

5. Produit : BlueWave Chromebook
   Catégorie : Computers and Laptops
   Marque : BlueWave
   Numéro du modèle : BW-CB100
   Garantie : 1 year
   Note : 4,1
   Caractéristiques : écran de 11"6, 4GB RAM, 32GB eMMC, Chrome OS
   Description : Un Chromebook compact et abordable pour les tâches quotidiennes.
   Prix : 249,99€

Étape 3:{delimiter} Si le message contient des produits dans la liste ci-dessus,
énumérez toutes les hypothèses que l'utilisateur fait dans son message,
par exemple que l'ordinateur portable X est plus grand que l'ordinateur portable Y,
ou que l'ordinateur portable Z a une garantie de 2 ans.

Étape 4:{delimiter} : Si l'utilisateur a fait des hypothèses, déterminez si 
l'hypothèse est vraie en vous basant sur vos informations sur les produits.

Étape 5:{delimiter} : D'abord, corrigez poliment les hypothèses incorrectes 
du client si applicable. Mentionnez ou faites référence uniquement aux produits 
dans la liste des 5 produits disponibles, car ce sont les seuls 5 produits que 
le magasin vend. Répondez au client avec un ton amical. 

Utilisez le format suivant :
Étape 1 :{delimiter} <step 1 reasoning>
Étape 2 :{delimiter} <step 2 reasoning>
Étape 3 :{delimiter} <step 3 reasoning>
Étape 4 :{delimiter} <step 4 reasoning>
Réponse pour l'utilisateur : {delimiter} <response to customer>

Assurez-vous d'inclure {delimiter} pour séparer chaque étape. 
"""

In [4]:
user_message = f"""De combien le Chromebook BlueWave 
est-il plus cher que le TechPro Desktop ?"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Étape 1 :#### L'utilisateur pose une question sur des produits spécifiques, en l'occurrence le Chromebook BlueWave et le TechPro Desktop.

Étape 2 :#### Les deux produits mentionnés, le BlueWave Chromebook et le TechPro Desktop, figurent dans la liste des produits disponibles.

Étape 3 :#### L'utilisateur suppose que le Chromebook BlueWave est plus cher que le TechPro Desktop.

Étape 4 :#### En vérifiant les prix, le BlueWave Chromebook coûte 249,99€, tandis que le TechPro Desktop coûte 999,99€. Donc, l'hypothèse de l'utilisateur est incorrecte.

Réponse pour l'utilisateur : #### En fait, le Chromebook BlueWave est moins cher que le TechPro Desktop. Le Chromebook coûte 249,99€, tandis que le TechPro Desktop est à 999,99€. Si vous avez d'autres questions sur ces produits ou si vous avez besoin d'aide pour choisir, n'hésitez pas à demander !


In [5]:
user_message = f"""
Est-ce que vous vendez des télés ?"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Étape 1 :#### L'utilisateur pose une question sur des produits en général, en demandant si des télés sont vendues.

Étape 2 :#### La question ne concerne pas un produit spécifique de la liste fournie.

Étape 3 :#### Il n'y a pas d'hypothèses à évaluer, car la question est générale.

Étape 4 :#### N/A

Réponse pour l'utilisateur : #### Je suis désolé, mais nous ne vendons pas de télés. Nous proposons uniquement des ordinateurs et des ordinateurs portables. Si vous avez des questions sur nos produits disponibles, n'hésitez pas à demander !


## Monologue interne
- Comme nous avons demandé au LLM de séparer ses étapes de raisonnement par un délimiteur, nous pouvons cacher le raisonnement en chaîne de la pensée dans la sortie finale que l'utilisateur voit.

In [6]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Désolé, nous rencontrons certains problèmes techniques, veuillez essayer de poser une autre question."
    
print(final_response)

Je suis désolé, mais nous ne vendons pas de télés. Nous proposons uniquement des ordinateurs et des ordinateurs portables. Si vous avez des questions sur nos produits disponibles, n'hésitez pas à demander !
